[Markdown Guide](https://docs.gitlab.com/ee/user/markdown.html)

# Configuration

| variable | type | description|
|:--|:--|:--|
|log_path | (path)| path to the XES File|
|use_networkx | (bool)| output via NetworkX or Pyvis|
|filter_timeframe | (bool)| filter the timeframe of the traces|
|filter_timeframe_intersecting | (bool)| use intersecting or contained|
|filter_timeframe_start | (dateTime)| start of the Timeframe|
|filter_timeframe_end | (dateTime)| end of the Timeframe|
|filter_performance| (bool)| filter on Traceperformance|
|filter_performance_min| (int)| minimum seconds of trace actions|
|filter_performance_max |(int)| maximum seconds of trace actions|

In [1]:
log_path = 'data/BPI_Challenge_2017_Offer_log.xes'

use_networkx = True

filter_timeframe = True
filter_timeframe_intersecting  = False
filter_timeframe_start = "2011-03-09 00:00:00"
filter_timeframe_end = "2011-03-09 00:00:00"

filter_performance = False
filter_performance_min = 0
filter_performance_max = 864000

Import libs used in the notebook

In [2]:
import shutil
from pm4py.objects.log.importer.xes import importer as xes_importer

from pm4py.algo.organizational_mining.sna import algorithm as sna
from pm4py.visualization.sna import visualizer as sna_visualizer
from pm4py.algo.organizational_mining.roles import algorithm as roles_discovery
from pm4py.algo.organizational_mining.sna import util
from pm4py.algo.filtering.log.timestamp import timestamp_filter
from pm4py.algo.filtering.log.cases import case_filter
import pm4py
import os
from pm4py.algo.organizational_mining.local_diagnostics import algorithm as local_diagnostics

Allgemeiner Importskript
inclusive filtering

In [3]:
log = xes_importer.apply(log_path)

if filter_timeframe:
    if not filter_timeframe_intersecting:
        log = timestamp_filter.filter_traces_contained(   log, filter_timeframe_start, filter_timeframe_end)
    else:
        log = timestamp_filter.filter_traces_intersecting(log, filter_timeframe_start, filter_timeframe_end)

if filter_performance:
    log = case_filter.filter_case_performance(log, filter_performance_min, filter_performance_max)

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (2453384756.py, line 4)

**Similar Activities**
Berechtet wie ähnlich die aktivitäten der Benutzer sind

Ergebnis: result/similar_activities.html

In [ ]:
sa_metric  = sna.apply(log, variant=sna.Variants.JOINTACTIVITIES_LOG)

if use_networkx:
    gviz_ja_py = sna_visualizer.apply(sa_metric , variant=sna_visualizer.Variants.NETWORKX)
    sna_visualizer.view(gviz_ja_py, variant=sna_visualizer.Variants.NETWORKX)
else:
    gviz_ja_py = sna_visualizer.apply(sa_metric , variant=sna_visualizer.Variants.PYVIS)
    shutil.move(gviz_ja_py, "result/similar_activities.html")

Discover Roles

In [ ]:
roles = roles_discovery.apply(log)
print([x for x in roles])

Handover of Work

In [ ]:
hw_values = sna.apply(log, variant=sna.Variants.HANDOVER_LOG)

if use_networkx:
    gviz_hw_py = sna_visualizer.apply(hw_values, variant=sna_visualizer.Variants.NETWORKX)
    sna_visualizer.view(gviz_hw_py, variant=sna_visualizer.Variants.NETWORKX)
else:
    gviz_hw_py = sna_visualizer.apply(sa_metric , variant=sna_visualizer.Variants.PYVIS)
    shutil.move(gviz_hw_py, "result/handover_of_work.html")

CLuster Similar Activities into groups
Clustering the results of the similar activities metric, individuals that work on the same tasks would be inserted in the same group.

In [ ]:
clustering_sa = util.cluster_affinity_propagation(sa_metric)
print(clustering_sa)

Clustering the results of the working together metric, individuals that work often together would be inserted in the same group.

In [ ]:
clustering_sa = util.cluster_affinity_propagation(hw_values)
print(clustering_sa)

In [ ]:
# this applies the organizational mining from an attribute that is in each event, describing the group that is performing the task.
ld = local_diagnostics.apply_from_group_attribute(log)
# GROUP RELATIVE FOCUS (on a given type of work) specifies how much a resource group performed this type of work
# compared to the overall workload of the group. It can be used to measure how the workload of a resource group
# is distributed over different types of work, i.e., work diversification of the group.
print("\ngroup_relative_focus")
print(ld["group_relative_focus"])
# GROUP RELATIVE STAKE (in a given type of work) specifies how much this type of work was performed by a certain
# resource group among all groups. It can be used to measure how the workload devoted to a certain type of work is
# distributed over resource groups in an organizational model, i.e., work participation by different groups.
print("\ngroup_relative_stake")
print(ld["group_relative_stake"])
# GROUP COVERAGE with respect to a given type of work specifies the proportion of members of a resource group that
# performed this type of work.
print("\ngroup_coverage")
print(ld["group_coverage"])
# GROUP MEMBER CONTRIBUTION of a member of a resource group with respect to the given type of work specifies how
# much of this type of work by the group was performed by the member. It can be used to measure how the workload
# of the entire group devoted to a certain type of work is distributed over the group members.
print("\ngroup_member_contribution")
print(ld["group_member_contribution"])